#### Task
Write a class that takes 2 set of 2d observations (pixel coordinates) and 2 sets of camera parameters and estimate the 3d coordinates of the observed points. 

In [1]:
import numpy as np
import cv2

In [2]:
class Triangulator:
    def __init__(self, K1, R1, t1, K2, R2, t2):
        """
        Kx: Intrinsic matrix (3x3)
        Rx: Rotation matrix (3x3)
        tx: Translation vector (3x1)
        """
        self.P1 = K1 @ np.hstack((R1, t1))
        self.P2 = K2 @ np.hstack((R2, t2))

    def triangulate(self, pts1, pts2):
        """
        pts1, pts2: Nx2 arrays of corresponding 2D points in image 1 and 2
        Returns: Nx3 array of 3D points
        """
        pts1 = pts1.T.astype(float)  # shape (2, N)
        pts2 = pts2.T.astype(float)

        pts_4d_h = cv2.triangulatePoints(self.P1, self.P2, pts1, pts2)  # shape (4, N)
        pts_3d = pts_4d_h[:3] / pts_4d_h[3]  # Convert from homogeneous to 3D
        return pts_3d.T  # shape (N, 3)


In [3]:
# Intrinsics
K = np.array([[800, 0, 320],
              [0, 800, 240],
              [0,   0,   1]])

# Camera extrinsics
R1 = np.eye(3)
t1 = np.zeros((3, 1))

R2 = cv2.Rodrigues(np.array([0, 0.05, 0]))[0]  # Small rotation around Y
t2 = np.array([[1.0], [0], [0]])  # Translate 1m on X axis

# Create synthetic 3D points
np.random.seed(42)
true_pts_3d = np.random.uniform(low=[-1, -1, 3], high=[1, 1, 5], size=(10, 3))

# Project points into both cameras
def project_points(pts_3d, K, R, t):
    rvec, _ = cv2.Rodrigues(R)
    image_pts, _ = cv2.projectPoints(pts_3d, rvec, t, K, None)
    return image_pts.squeeze()

img_pts1 = project_points(true_pts_3d, K, R1, t1)
img_pts2 = project_points(true_pts_3d, K, R2, t2)

# Triangulate
triang = Triangulator(K, R1, t1, K, R2, t2)
reconstructed_pts_3d = triang.triangulate(img_pts1, img_pts2)

# Step 6: Compare results
print("True 3D Points:\n", true_pts_3d)
print("\nReconstructed 3D Points:\n", reconstructed_pts_3d)
print("\nReconstruction Error (per point):")
print(np.linalg.norm(true_pts_3d - reconstructed_pts_3d, axis=1))
print("\nMean Error:", np.mean(np.linalg.norm(true_pts_3d - reconstructed_pts_3d, axis=1)))

True 3D Points:
 [[-0.25091976  0.90142861  4.46398788]
 [ 0.19731697 -0.68796272  3.31198904]
 [-0.88383278  0.73235229  4.20223002]
 [ 0.41614516 -0.95883101  4.9398197 ]
 [ 0.66488528 -0.57532178  3.36364993]
 [-0.63319098 -0.39151551  4.04951286]
 [-0.13610996 -0.41754172  4.22370579]
 [-0.72101228 -0.4157107   3.73272369]
 [-0.08786003  0.57035192  3.39934756]
 [ 0.02846888  0.18482914  3.09290083]]

Reconstructed 3D Points:
 [[-0.25091976  0.90142861  4.46398788]
 [ 0.19731697 -0.68796272  3.31198904]
 [-0.88383278  0.73235229  4.20223002]
 [ 0.41614516 -0.95883101  4.9398197 ]
 [ 0.66488528 -0.57532178  3.36364993]
 [-0.63319098 -0.39151551  4.04951286]
 [-0.13610996 -0.41754172  4.22370579]
 [-0.72101228 -0.4157107   3.73272369]
 [-0.08786003  0.57035192  3.39934756]
 [ 0.02846888  0.18482914  3.09290083]]

Reconstruction Error (per point):
[1.45388357e-14 8.22858031e-15 1.28848067e-14 1.53765889e-14
 8.61264054e-15 1.17032463e-14 1.16135113e-14 9.52594646e-15
 9.02636753e-15 8